## Title: Managed Forest Concessions

### Description
Displays boundaries of forested areas allocated by governments to companies for harvesting timber and other wood products. <br>

Logging concession refers to an area allocated by a government for logging in a public forest. Logging concessions are distinct from wood fiber concessions, where plantation forests are established for the production of pulp and paper products. Concession is used as a general term for licenses, permits, or other contracts that confer rights to private companies to manage and extract timber from public forests; terminology varies at the national level, however, and includes "forest permits," "tenures," "licenses," and other terms. The logging concession data on GFW, while displayed as a single layer, is assembled on a country-by-country basis from multiple sources. Logging concession data displayed on the GFW website vary from country to country by date and data sources. Data may come from government agencies, NGOs, or other organizations. <br>

This data set displays managed forest concessions as a single layer assembled by aggregating data for multiple countries. The data may come from government agencies, NGOs, or other organizations and varies by date and data sources. For more information on concession data for each country please visit the Global Forest Watch Open Data Portal [link](https://data.globalforestwatch.org/datasets/managed-forest-concessions-1).

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/

### Format
<b>Extent: </b>Canada (CAN), Liberia (LBR), Malaysia (MYS), Indonesia (IND), Central African Republic (CAF), Cameroon (CMR), Democratic Republic of the Congo (COD), Republic of the Congo (COG), Gabon (GAB), Equatorial Guinea (GNQ)<br>
<b>Format</b>: Vector polyon geoJSON<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Year: Mixed</b><br>
<b>Size:</b>  

### Original source
From: https://data.globalforestwatch.org/datasets/managed-forest-concessions-1<br>
Direct download: https://gfw2-data.s3.amazonaws.com/forest_use/logging/zip/gfw_logging_download.zip<br>
Shapefile, polygon

### Licence
Dataset has too many sub-components so overarching licence is not possible. Please cite the source and any changes made.

### Citation
“Managed forest concessions.” Accessed through Global Forest Watch www.globalforestwatch.org 29/12/2020.

Credits (Attribution)
World Resources Institute; Global Forest Watch Canada; Cameroon Ministry of Forestry and Wildlife; Central African Republic Ministry of Water and Forests, Hunting, and Fishing; German Technical Cooperation (GIZ); French Development Agency (AFD); Special Allocation Fund for Forest Development (CASDF); Democratic Republic of the Congo Ministry of Environment, Nature Conservation, and Tourism (MECNT); Department for Permanent Service for Forest Inventory and Management (SPIAF); Asset Management and Forest Management (DIAF); Forest Management Branch (FMB); Equatorial Guinea Ministry of Agriculture and Forests; Gabon Ministry of Forest Economy, Water, Fisheries, and Aquaculture (MEFEPA); Indonesia Ministry of Forestry; Global Witness, 2012; Republic of the Congo Ministry of Forest Economy (MEF), National Center for Inventory and Planning of Forest and Wildlife Resources (CNIAF)


### Metadata
Displays boundaries of forested areas allocated by governments to companies for harvesting timber and other wood products.  Data only available for Canada (CAN), Liberia (LBR), Malaysia (MYS), Indonesia (IND), Central African Republic (CAF), Cameroon (CMR), Democratic Republic of the Congo (COD), Republic of the Congo (COG), Gabon (GAB), Equatorial Guinea (GNQ). These are processed to appear together and apart.<br>

View https://data.globalforestwatch.org/datasets/managed-forest-concessions-1

### Notes
This layer is a compilation of concession data from various countries and sources. The quality of these data can vary depending on the source. This layer may not be comprehensive of all existing concessions in a country, and the location of certain concessions can be inaccurate. There is some topological error that requires fixing.

### Processing
See below for arcpy code to fix geometry, gaps and overlaps and convert to JSON. Individual country data have also been separated out and are available in the data library.

In [ ]:
import arcpy
import os

# Input variables
in_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\ManagedForestConcessions"
scr_folder = os.path.join(in_folder, "scratch.gdb")
out_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\ManagedForestConcessions\json"
fullfield = "country" #this needs to be a field in the original table that is fully populated
smallest = "20000" #smallest area to be fixed in m2 - gaps and slithers
scr = arcpy.CreateFileGDB_management(out_folder, "scratch")
scr_folder = os.path.join(out_folder, "scratch.gdb")

# Environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True
field = fullfield + " IS NULL or " + fullfield + " = ''"
arcpy.env.parallelProcessingFactor = "100%"

# List features to process
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Repair/check topology and make FLINT ready
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    whereclause = "FID_" + fcname + " =-1 AND AREA_GEO <= " + smallest
    print(fcname + ' processing...')
    
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]
    arcpy.management.DeleteIdentical(fLayer, "Shape")
    
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(projectSingle, dissolveSlither, None, None,"SINGLE_PART")
    
    # Take action if no overlaps
    if arcpy.management.GetCount(dissolveSlither)[0] == "0":
        print('no overlaps detected...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(fLayer,projectUnion, "ALL", None, "NO_GAPS")
        
        unionSingle = os.path.join(scr_folder, "unionSingle")
        arcpy.management.MultipartToSinglepart(projectUnion, unionSingle)
        
        arcpy.management.DeleteIdentical(unionSingle, "Shape")
        arcpy.management.AddGeometryAttributes(unionSingle, "AREA_GEODESIC", None, "SQUARE_METERS")
        
        
        uniSelect = os.path.join(scr_folder, "uniSelect")
        arcpy.analysis.Select(unionSingle, uniSelect, whereclause)
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
                     
            # Progress report no error
            print(fcname, 'No gaps and overlaps. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            print('gaps detected')

            appendGap = arcpy.management.Append(uniSelect, fLayer, "NO_TEST")     
            selectGap = arcpy.management.SelectLayerByAttribute(fLayer, "NEW_SELECTION", field)

            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")

            # Progress report 
            print(fcname, 'No overlaps but gaps detected and repaired. Repairing geometry and conversion to json...')

            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")


    else:
        print('Overlaps detected...')
        # Fix overlaps
        projectErase = os.path.join(scr_folder, "projectErase")
        arcpy.analysis.Erase(fLayer, dissolveSlither, projectErase)
        
        arcpy.management.Append(dissolveSlither, projectErase, "NO_TEST")
    
        selectSlither = arcpy.management.SelectLayerByAttribute(projectErase, "NEW_SELECTION",  field)
        
        eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
        arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
        
        print('Overlaps detected and fixed...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(eliminateSlither, projectUnion, "ALL", None, "NO_GAPS")
        
        unionSingle = os.path.join(scr_folder, "unionSingle")
        arcpy.management.MultipartToSinglepart(projectUnion, unionSingle)
        arcpy.management.DeleteIdentical(unionSingle, "Shape")
                                         
        arcpy.management.AddGeometryAttributes(unionSingle, "AREA_GEODESIC", None, "SQUARE_METERS")
        
        uniSelect = os.path.join(scr_folder, "uniSelect")
        whereUnion= "FID_eliminateSlither = -1 AND AREA_GEO <= " + smallest
        arcpy.analysis.Select(unionSingle, uniSelect, whereUnion)
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
            
            # Progress report no error
            print(fcname, ' No gaps detected. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            appendGap = arcpy.management.Append(uniSelect, eliminateSlither, "NO_TEST")
            
            selectGap = arcpy.management.SelectLayerByAttribute(eliminateSlither, "NEW_SELECTION",  field)
            
            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")
            
            print('gaps detected and repaired')
            
            # Progress report
            print(fcname, 'Gaps and overlaps fixed. Repairing geometry and conversion to json...')
            
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]
            
            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            print(outjson)
            
arcpy.AddMessage("All done!")
print('done')